In [1]:
# Spark Session, Pipeline, Functions, and Metrics
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SQLContext
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
# from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# # Keras / Deep Learning
# from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation
# from keras import optimizers, regularizers
# from keras.optimizers import Adam

# # Elephas for Deep Learning on Spark
# from elephas.ml_model import ElephasEstimator

# time 
import time

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1588575344103_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package("pandas")
import pandas

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# sc.stop()

In [ ]:
# Spark Session
# run locally with 4 word nodes
# conf = SparkConf().setAppName('678 Spark Pipeline').setMaster('local[4]') 
# sc = SparkContext()
# sql_context = SQLContext(sc)

In [3]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [4]:
# Load Data to Spark Dataframe
path = './data/simplified_train.csv'
path = "s3://shelly-678-project/simplified_train.csv"
df = spark.read.csv(path, header=True, inferSchema=True)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Change name for label column
label = 'returnsOpenNextMktres10'
df = df.withColumnRenamed(label,'label')

# Shuffle Data
df = df.orderBy(rand())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
<b>Step 4:</b> Create the Spark Data Pipeline
</div>

In [6]:
def create_data_pipeline():
    # Spark Pipeline
    label = 'returnsOpenNextMktres10'
    cat_feature = 'assetCode'
    num_features = ['volume',
                     'close',
                     'open',
                     'returnsClosePrevRaw1',
                     'returnsOpenPrevRaw1',
                     'returnsClosePrevMktres1',
                     'returnsOpenPrevMktres1',
                     'returnsClosePrevRaw10',
                     'returnsOpenPrevRaw10',
                     'returnsClosePrevMktres10',
                     'returnsOpenPrevMktres10',
                     'urgency',
                     'takeSequence',
                     'marketCommentary',
                     'companyCount',
                     'relevance',
                     'bodySize',
                     'sentenceCount',
                     'wordCount',
                     'firstMentionSentence',
                     'sentimentClass',
                     'sentimentNegative',
                     'sentimentNeutral',
                     'sentimentPositive',
                     'sentimentWordCount',
                     'noveltyCount12H',
                     'noveltyCount24H',
                     'noveltyCount3D',
                     'noveltyCount5D',
                     'noveltyCount7D',
                     'volumeCounts12H',
                     'volumeCounts24H',
                     'volumeCounts3D',
                     'volumeCounts5D',
                     'volumeCounts7D']
    # Pipeline Stages List
    stages = []
    
    # Loop for StringIndexer and OHE for Categorical Variables
    string_indexer = StringIndexer(inputCol=cat_feature, outputCol=cat_feature + "_index").setHandleInvalid("keep")
    encoder = OneHotEncoderEstimator(inputCols=[string_indexer.getOutputCol()],
                                     outputCols=[cat_feature + "_class_vec"])
    stages += [string_indexer, encoder]

    # Scale Feature
    unscaled_assembler = VectorAssembler(inputCols=num_features, outputCol="unscaled_features")
    scaler = StandardScaler(inputCol="unscaled_features", outputCol="scaled_features")
    stages += [unscaled_assembler, scaler]


    # Assemble or Concat the Categorical Features and Numeric Features
#     assembler_inputs = [feature + "_class_vec" for feature in cat_features]
#     assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="assembled_inputs") 
#     stages += [assembler]

    # Assemble Final Training Data of Scaled, and Categorical Engineered Features
    assembler_final = VectorAssembler(inputCols=["scaled_features",cat_feature + "_class_vec"], outputCol="features")
    stages += [assembler_final]
    print("Stages for data transformation pipeline:\n",stages)
    # Set Pipeline
    pipeline = Pipeline(stages=stages)
    return pipeline


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
<b>Step 7:</b> Build a Generalized Linear Regression
</div>

In [7]:
def training(df, percentage, model_name):
    """
    main func of the program
    """
    start = time.time()

    # get exact percentage of data
    if percentage < 1:
        df, df_rest = df.randomSplit([percentage, (1-percentage)],seed=1234)
    print("Using {} % of the total dataset".format(percentage*100))
    # create data processing pipeline
    pipeline = create_data_pipeline()
    print("Created data processing pipeline")
    # Split Data into Train / Test Sets
    train_data, test_data = df.randomSplit([.8, .2],seed=1234)
    
    # Fit Pipeline to Data
    pipeline_model = pipeline.fit(train_data)
    
    # Transform Data using Fitted Pipeline
    train_data = pipeline_model.transform(train_data)
    test_data = pipeline_model.transform(test_data)
    
    # select final columns
    train_data = train_data.select('features','label')
    test_data =test_data.select('features','label')
#     print(train_data.limit(5).toPandas())
#     return train_data, test_data
    if model_name == "GLR":
        # builde model
        glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)    
        # training
        model = glr.fit(train_data)

    summary = model.summary

    # make predictions
    predictions = model.transform(test_data)

    # evaluate the result
    evaluator1 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
    rmse = evaluator1.evaluate(predictions)
    print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

    evaluator2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
    mse = evaluator2.evaluate(predictions)
    print("Mean Squared Error (MSE) on test data = %g" % mse)

    # time calculation
    end = time.time()
    total_time = end - start
    print("Total processing time : ", total_time)
    print()

    return predictions, summary, (percentage, rmse, mse, total_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
percentages = [.2, .4, .8, 1.0]
predictions_list = []
summary_list = []
result_list = []
for i in percentages:
    predictions, summary, result = training(df, i,"GLR")
    predictions_list.append(predictions)
    summary_list.append(summary)
    result_list.append(result)
print("All Done!")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Using 20.0 % of the total dataset
Stages for data transformation pipeline:
 [StringIndexer_9a24dc5d1433, OneHotEncoderEstimator_3d1883612403, VectorAssembler_65dcfebec77e, StandardScaler_10eef180dab3, VectorAssembler_5089d0fc16d4]
Created data processing pipeline
Root Mean Squared Error (RMSE) on test data = 0.0638285
Mean Squared Error (MSE) on test data = 0.00407408
Total processing time :  203.51048302650452

Using 40.0 % of the total dataset
Stages for data transformation pipeline:
 [StringIndexer_fd250fa2be8a, OneHotEncoderEstimator_16abef6b60ed, VectorAssembler_951d16ede031, StandardScaler_9a9edfcd6148, VectorAssembler_cf410e3151c6]
Created data processing pipeline
Root Mean Squared Error (RMSE) on test data = 0.0637236
Mean Squared Error (MSE) on test data = 0.0040607
Total processing time :  197.68755865097046

Using 80.0 % of the total dataset
Stages for data transformation pipeline:
 [StringIndexer_6bbfe5ed29d9, OneHotEncoderEstimator_fbeb953a1633, VectorAssembler_9605d561962

In [9]:
result_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(0.2, 0.06382852222837014, 0.004074080249857543, 203.51048302650452), (0.4, 0.06372361725251827, 0.0040606993957454434, 197.68755865097046), (0.8, 0.06404456217233037, 0.004101705943845489, 203.95586252212524), (1.0, 0.06397686500683773, 0.0040930392561031375, 195.0188548564911)]

In [10]:
result_df = spark.createDataFrame(result_list,('percentage', 'rmse','mse','time'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
result_df.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   percentage      rmse       mse        time
0         0.2  0.063829  0.004074  203.510483
1         0.4  0.063724  0.004061  197.687559
2         0.8  0.064045  0.004102  203.955863
3         1.0  0.063977  0.004093  195.018855

In [ ]:
result_df.write.csv('s3n://shelly-678-project/1-master-3-work')

# "s3://shelly-678-project/simplified_train.csv"


<div class="alert alert-block alert-info">
<b>Step 7:</b> Build a Deep Learning Model
</div>